# finding probability based on Tournament Performance

### finding slop

In [ ]:
def find_slop(players, round_completed):
    for player in players:
        player['slop'] = (player['updated_elo'] - player['base_elo']) / (round_completed - 0)        

### distributing k-factor

In [ ]:
def k_factor_allocation(player):
    # dividing players into three groups
    # FINDING LIMITS
    
    # find lowest and highest base_elo
    min_elo = players["base_elo"].min()
    max_elo = players["base_elo"].max()
    
    W = (max_elo - min_elo) / 3
    limit1 = min_elo + W
    limit2 = min_elo + 2*W
    limit3 = min_elo + 3*W
    
    for player in players:
        if player["base_elo"] <= limit1:
            player["k_factor"] = 40
        else if player["base_elo"] <= limit2:
            player["k_factor"] = 20
        else if layer["base_elo"] <= limit3:
            player["k_factor"] = 10
        

In [7]:
def curr_tournament_performance():
    for player in players:
        player['curr_form'] = player['k_factor'] * player['slop']
    # final slop, add a column for all players       

### After actual match result 

In [ ]:
# this is to update in Elo rating after each match
def calculate_win_probability_for_a_match(player1, player2): 
    adjusted_elo1 = player1['updated_elo']
    adjusted_elo2 = player2['updated_elo']
    return 1 / (1 + 10 ** ((adjusted_elo2 - adjusted_elo1) / 400))

def update_elo_ratings(winner, loser, draw=False):
    """ Update Elo ratings and current form after a game """
    expected_win_winner = calculate_win_probability_for_a_match(winner, loser)
    expected_win_loser = calculate_win_probability_for_a_match(loser, winner)

    score_winner = 0.5 if draw else 1
    score_loser = 0.5 if draw else 0

    winner['updated_elo'] += player['k_factor'] * (score_winner - expected_win_winner)
    loser['updated_elo'] += player['k_factor'] * (score_loser - expected_win_loser)

###  Finding win probability(tournament) for each player

In [9]:
#  This is to find chances in upcoming tournament matches
def calculate_win_probability_for_tournament(players): 
    for player in players:
        player['points'] = player['updated_elo'] + player['curr_form']
        
    chance_to_win_a_tournament(players)

In [13]:
def chance_to_win_a_tournament(players):
    pass
    # based on curr_tournament_performance output, add column
    

In [15]:
def swiss_pairing(players):
    """ Pair players using a simple Swiss system approach """
    sorted_players = sorted(players, key=lambda x: x['points'], reverse=True)
    pairings = []
    while sorted_players:
        player1 = sorted_players.pop(0)
        for i, player2 in enumerate(sorted_players):
            if not 'opponents' in player1 or player2['name'] not in player1['opponents']:
                player1.setdefault('opponents', []).append(player2['name'])
                player2.setdefault('opponents', []).append(player1['name'])
                pairings.append((player1, player2))
                sorted_players.pop(i)
                break
    return pairings

In [16]:
# curr_round_is_given as an input
# curr_round = 2
# def simulate_tournament():
#      for round_number in range(curr_round, num_rounds + 1):
#         pairings = swiss_pairing(players)
#         for player1, player2 in pairings:
#             simulate_game(player1, player2)


In [17]:
def simulate_game(player1, player2):
    win_prob = calculate_win_probability_for_a_match(player1, player2)
    pass


In [ ]:
def simulate_

### Updated code Dec 04

In [ ]:
import random

# Number of Players, Rounds, and Simulations
num_players = 10
num_total_rounds = 9
rounds_completed = 5
num_simulations = 1000
k_factor = 10  # Higher K-factor for more responsive Elo rating updates

# Initialize Players with Elo ratings and initial tournament points
initial_points = [4, 3, 2.5, 2, 1.5, 1, 1, 0.5, 0, 0]
initial_elo = [random.randint(1200, 2000) for i in range(num_players)]

# calculate curr_form based on initial_points and k-factor


players = [{'name': f'Player {i}', 'elo': initial_elo[i], 'simulation_points': initial_points[i], 'wins': 0, 'current_form': 0} for i in range(num_players)]

for player in players:
    print(f"{player['name']} - Init Elo: {player['elo']:.2f}, Init Points: {player['simulation_points']}")

def calculate_win_probability(player1, player2, rounds_completed):
    """ Calculate the probability of player1 winning against player2, considering current form and total points after x rounds """
    adjusted_elo1 = player1['elo'] + player1['current_form']
    adjusted_elo2 = player2['elo'] + player2['current_form']

    # Consider total points gained after x rounds
    points_diff = player1['simulation_points'] - player2['simulation_points']

    return 1 / (1 + 10 ** ((adjusted_elo2 - adjusted_elo1 + points_diff) / 400))

def update_elo_ratings(winner, loser, draw=False):
    """ Update Elo ratings and current form after a game """
    expected_win_winner = calculate_win_probability(winner, loser, rounds_completed)
    expected_win_loser = calculate_win_probability(loser, winner, rounds_completed)

    score_winner = 0.5 if draw else 1
    score_loser = 0.5 if draw else 0

    winner['elo'] += k_factor * (score_winner - expected_win_winner)
    loser['elo'] += k_factor * (score_loser - expected_win_loser)

    # Remove this calculation from here
    winner['current_form'] += k_factor * 1.2 if not draw else -k_factor * 0.3
    loser['current_form'] += -k_factor * 1.2 if not draw else k_factor * 0.5

def simulate_game(player1, player2):
    """ Simulate a game between two players """
    win_prob = calculate_win_probability(player1, player2, rounds_completed)
    result = random.random()
    if result < win_prob:
        player1['simulation_points'] += 1  # Player 1 wins
        update_elo_ratings(player1, player2)
    elif result < win_prob + (1 - win_prob) / 2:
        player1['simulation_points'] += 0.5
        player2['simulation_points'] += 0.5  # Draw
        update_elo_ratings(player1, player2, draw=True)
    else:
        player2['simulation_points'] += 1  # Player 2 wins
        update_elo_ratings(player2, player1)

# Simulate remaining rounds
for round_number in range(rounds_completed + 1, num_total_rounds + 1):
    pairings = swiss_pairing(players)
    for player1, player2 in pairings:
        simulate_game(player1, player2)
    
# Print final probabilities after completing the tournament
print("\nFinal Tournament Results after all simulations:")
for player in players:
    winning_probability = player['wins'] / num_simulations
    print(f"{player['name']} - - Points: {player['simulation_points']} - - Final Elo: {player['elo']:.2f} - - Winning Probability: {winning_probability:.2f}")


### Last hope --> Successful

In [59]:
import random

num_players = 24
num_total_rounds = 9
rounds_completed = 5
num_simulations = 2500

# Initialize Players with initial tournament points
# initial_points = [4, 3, 2.5, 2, 1.5, 1, 3, 0.5, 0, 0, ]
initial_points = [4.0, 3.5, 1.0, 2.0, 4, 3.5, 2.5, 3.0, 3.0, 4, 1.5, 1, 2.0, 3, 0, 4.5, 2.5, 3.0, 3.0, 2.0, 2.0, 1.5, 2.0, 2.5]


players = [{'name': f'Player {i}', 'elo': 0, 'simulation_points': initial_points[i], 'k_factor': 0, 'wins': 0, 'current_form': 0} for i in range(num_players)]
elo_values = [2834,2590,2747,2712,2628,2741,2727,2802,2628,2676,2578,2708,2791,2766,2659,2629,2656,2759,2777,2646,2548,2442,2534,2743,2539,2526,2746]

for i, player in enumerate(players):
    player['elo'] = elo_values[i]

# Sort players by rating
# players.sort(key=lambda x: x['elo'])

def k_factor_allocation(players):
    # dividing players into three groups
    # FINDING LIMITS
    
    # find lowest and highest base_elo
    min_elo = min(elo_values)
    max_elo = max(elo_values)
    
    W = (max_elo - min_elo) / 3
    limit1 = min_elo + W
    limit2 = min_elo + 2*W
    limit3 = min_elo + 3*W
    
    for player in players:
        if player["elo"] <= limit1:
            player["k_factor"] = 40
        elif player["elo"] <= limit2:
            player["k_factor"] = 20
        elif player["elo"] <= limit3:
            player["k_factor"] = 10

k_factor_allocation(players)

# Divide players into three groups
# third_of_players = len(players) // 3
# low_rating_group = players[:third_of_players]
# mid_rating_group = players[third_of_players:2*third_of_players]
# high_rating_group = players[2*third_of_players:]

# k_factors = [40, 20, 10]  # Higher K-factor for lower-rated players

# for player in low_rating_group:
#     player['k_factor'] = k_factors[0]

# for player in mid_rating_group:
#     player['k_factor'] = k_factors[1]

# for player in high_rating_group:
#     player['k_factor'] = k_factors[2]

    

# calculate curr_form based on initial_points and k-factor
for player in players:
    player['current_form'] = player['k_factor'] * ((player['simulation_points'] ** 2) / rounds_completed ) #- sum(initial_points) / num_players

for player in players:
    print(f"{player['name']} - Init Elo: {player['elo']:.2f}, Init Points: {player['simulation_points']}, curr_form: {player['current_form']}, k_factor: {player['k_factor']}, sum: {player['current_form']+player['elo']}")

def calculate_win_probability(player1, player2):
    """ Calculate the probability of player1 winning against player2, considering current form and total points after x rounds """
    adjusted_elo1 = player1['elo'] + player1['current_form']
    adjusted_elo2 = player2['elo'] + player2['current_form']

    return 1 / (1 + 10 ** ((adjusted_elo2 - adjusted_elo1) / 400))

def update_elo_ratings(winner, loser, draw=False):
    """ Update Elo ratings and current form after a game """
    expected_win_winner = calculate_win_probability(winner, loser)
    expected_win_loser = calculate_win_probability(loser, winner)

    score_winner = 0.5 if draw else 1
    score_loser = 0.5 if draw else 0

    winner['elo'] += winner['k_factor'] * (score_winner - expected_win_winner)
    loser['elo'] += loser['k_factor'] * (score_loser - expected_win_loser)
    

def simulate_game(player1, player2, round_number):
    """ Simulate a game between two players """
    win_prob = calculate_win_probability(player1, player2)
    result = random.random()
    if result < win_prob:
        player1['simulation_points'] += 1  # Player 1 wins
        update_elo_ratings(player1, player2)
        player1['current_form'] = player1['k_factor'] * ((player1['simulation_points'] ** 2) / rounds_completed )
    elif result < win_prob + (1 - win_prob) / 2:
        player1['simulation_points'] += 0.5
        player2['simulation_points'] += 0.5  # Draw
        update_elo_ratings(player1, player2, draw=True)
    else:
        player2['simulation_points'] += 1  # Player 2 wins
        player2['k_factor'] * ((player2['simulation_points'] ** 2) / rounds_completed )
        update_elo_ratings(player2, player1)
        
def swiss_pairing(players):
    
    sorted_players = sorted(players, key=lambda x: x['simulation_points'], reverse=True)
    pairings = []
    while sorted_players:
        player1 = sorted_players.pop(0)
        for i, player2 in enumerate(sorted_players):
            if not 'opponents' in player1 or player2['name'] not in player1['opponents']:
                player1.setdefault('opponents', []).append(player2['name'])
                player2.setdefault('opponents', []).append(player1['name'])
                pairings.append((player1, player2))
                sorted_players.pop(i)
                break
    return pairings

def simulate_tournament():
    
    # Reset points and opponents for each player for the new tournament
    for player in players:
        player['simulation_points'] = initial_points[players.index(player)]
        player['opponents'] = []
        player['current_form'] = player['k_factor'] * ((player['simulation_points'] ** 2) / rounds_completed )
        player['elo'] = elo_values[players.index(player)]

    # Simulate remaining rounds
    for round_number in range(rounds_completed + 1, num_total_rounds + 1):
        pairings = swiss_pairing(players)
        for player1, player2 in pairings:
            simulate_game(player1, player2, round_number)

    # Determine tournament winner and update wins
    tournament_winner = max(players, key=lambda x: x['simulation_points'])
    tournament_winner['wins'] += 1

# Reset wins count before all simulations
for player in players:
    player['wins'] = 0
    
# Simulate all tournaments
for _ in range(num_simulations):
    simulate_tournament()

# Sort players by final Elo rating before printing the results
# players.sort(key=lambda x: x['elo'], reverse=True)

# Print final probabilities after completing the simulations
print("\nFinal Tournament Results after all simulations:")
for player in players:
    winning_probability = player['wins'] / num_simulations
    print(f"{player['name']} - Points: {player['simulation_points']} - Final Elo: {player['elo']:.0f} - Wins: {player['wins']} - Winning Probability: {winning_probability:.3f}")
   

Player 0 - Init Elo: 2834.00, Init Points: 4.0, curr_form: 32.0, k_factor: 10, sum: 2866.0
Player 1 - Init Elo: 2590.00, Init Points: 3.5, curr_form: 49.0, k_factor: 20, sum: 2639.0
Player 2 - Init Elo: 2747.00, Init Points: 1.0, curr_form: 2.0, k_factor: 10, sum: 2749.0
Player 3 - Init Elo: 2712.00, Init Points: 2.0, curr_form: 8.0, k_factor: 10, sum: 2720.0
Player 4 - Init Elo: 2628.00, Init Points: 4, curr_form: 64.0, k_factor: 20, sum: 2692.0
Player 5 - Init Elo: 2741.00, Init Points: 3.5, curr_form: 24.5, k_factor: 10, sum: 2765.5
Player 6 - Init Elo: 2727.00, Init Points: 2.5, curr_form: 12.5, k_factor: 10, sum: 2739.5
Player 7 - Init Elo: 2802.00, Init Points: 3.0, curr_form: 18.0, k_factor: 10, sum: 2820.0
Player 8 - Init Elo: 2628.00, Init Points: 3.0, curr_form: 36.0, k_factor: 20, sum: 2664.0
Player 9 - Init Elo: 2676.00, Init Points: 4, curr_form: 64.0, k_factor: 20, sum: 2740.0
Player 10 - Init Elo: 2578.00, Init Points: 1.5, curr_form: 9.0, k_factor: 20, sum: 2587.0
Playe

## Simulation score tracking for player 6